In [ ]:
import pandas as pd
from os import walk
import re

data_dir = 'data'
quarterly_valuation_measures = 'quarterly_valuation_measures'
quarterly_income_statement = 'quarterly_financials'
quarterly_cash_flow = 'quarterly_cash-flow'
quarterly_balance_sheet = 'quarterly_balance-sheet'

def get_stock_tickers(dir):
	filenames = next(walk(dir), (None, None, []))[2]  # [] if no file
	filenames = [re.sub('[_.].+$', '', f) for f in filenames if re.match('[A-Z]+_.+\.csv', f)]
	return list(set(filenames))

def combine_financials(data_dir, stock, financials):
	df = pd.concat([pd.read_csv('{}/{}_{}.csv'.format(data_dir, stock, financial)) for financial in financials])
	df = df[df['name'].str.match('^\t.*') == False]
	df = df.replace({',': ''}, regex=True)
	df = df.set_index('name').T
	for col in df.columns:
		df[col] = pd.to_numeric(df[col], errors='ignore')
	df.reset_index(inplace=True)
	df = df.rename(columns = {'index':'Quarter'})
	return df

def calculate_ratios(df):
	measures = [
		('GrossMargin', 'GrossProfit', 'TotalRevenue', 100),
		('OperatingMargin', 'OperatingIncome', 'TotalRevenue', 100),
		('NetMargin', 'PretaxIncome', 'TotalRevenue', 100),
		('EV2GrossIncome', 'EnterpriseValue', 'GrossProfit', 1),
		('EV2OperatingIncome', 'EnterpriseValue', 'OperatingIncome', 1),
		('EV2NetIncome', 'EnterpriseValue', 'PretaxIncome', 1),
		('EV2FreeCashFlow', 'EnterpriseValue', 'FreeCashFlow', 1),
		('EV2OperatingCashFlow', 'EnterpriseValue', 'OperatingCashFlow', 1),
		('EV2TotalRevenue', 'EnterpriseValue', 'TotalRevenue', 1),
	]

	for m in measures:
		ratio, numerator, denomitor, scale = m
		if not (numerator in df.columns and denomitor in df.columns):
			continue
		
		df[ratio] = None

		for i in range(0, len(df[denomitor])):
			a = df.at[i, numerator]
			b = df.at[i, denomitor]
			if not (pd.isna(a) or pd.isna(b) or b == 0):
				df.at[i, ratio] = (a / b) * scale
	return df

def calculate_change_over_period(df):
	measures = [
		('YoY', 'InterestIncome'),
		('YoY', 'NormalizedIncome'),
		('YoY', 'DilutedNIAvailtoComStockholders'),
		('YoY', 'NetIncomeCommonStockholders'),
		('YoY', 'PretaxIncome'),
		('YoY', 'TotalOperatingIncomeAsReported'),
		('YoY', 'OperatingIncome'),
		('QoQ', 'OperatingIncome'),
		('YoY', 'EBIT'),
		('QoQ', 'EBIT'),
		('YoY', 'FreeCashFlow'),
		('QoQ', 'FreeCashFlow'),
		('YoY', 'NormalizedEBITDA'),
		('YoY', 'EBITDA'),
		('QoQ', 'EBITDA'),
		('YoY', 'OperatingCashFlow'),
		('QoQ', 'OperatingCashFlow'),
		('YoY', 'OperatingExpense'),
		('QoQ', 'OperatingExpense'),
		('YoY', 'ReconciledCostOfRevenue'),
		('YoY', 'CostOfRevenue'),
		('QoQ', 'CostOfRevenue'),
		('YoY', 'GrossProfit'),
		('QoQ', 'GrossProfit'),
		('YoY', 'EndCashPosition'),
		('QoQ', 'EndCashPosition'),
		('YoY', 'TotalExpenses'),
		('QoQ', 'TotalExpenses'),
		('YoY', 'TotalRevenue'),
		('QoQ', 'TotalRevenue'),
		('YoY', 'MarketCap'),
		('QoQ', 'MarketCap'),
		('YoY', 'EnterpriseValue'),
		('QoQ', 'EnterpriseValue')
	]

	for m in measures:
		period, measure = m
		if measure not in df.columns:
			continue
		
		change_over_period_measure = measure + period
		df[change_over_period_measure] = None

		if period == 'YoY':
			offset = 4
		else:
			offset = 1

		for i in range(1, len(df[measure]) - 1 - offset):
			current = df.at[i, measure]
			previous = df.at[i + offset, measure]
			if not (pd.isna(current) or pd.isna(previous) or previous == 0):
				df.at[i, change_over_period_measure] = ((current - previous) / previous) * 100
	return df


In [ ]:
import pandas as pd

today = '06/20/2022'
quarterly_financials = [quarterly_valuation_measures, quarterly_income_statement, quarterly_balance_sheet, quarterly_cash_flow]

all_df = pd.DataFrame()
for stock in get_stock_tickers(data_dir):
	print(stock)
	df = combine_financials(data_dir, stock, quarterly_financials)
	df = calculate_change_over_period(df)
	df = calculate_ratios(df)
	df['Ticker'] = stock
	all_df = pd.concat([all_df, df], ignore_index=True)

	df.to_csv('{}/{}_quarterly_combined.csv'.format(data_dir, stock), index=False)
all_df.loc[all_df["Quarter"] == "ttm", "Quarter"] = today
all_df.to_csv('{}/all_quarterly_combined.csv'.format(data_dir), index=False)


In [ ]:
stock = 'SNOW'
df = combine_financials(data_dir, stock, quarterly_financials)
df = calculate_YoY(df)

measures = [
	# 'InterestIncome',
	# 'NormalizedIncome',
	# 'DilutedNIAvailtoComStockholders',
	# 'NetIncomeCommonStockholders',
	# 'PretaxIncome',
	# 'TotalOperatingIncomeAsReported',
	# 'OperatingIncome',
	# 'EBIT',
	# 'FreeCashFlow',
	# 'NormalizedEBITDA',
	# 'EBITDA',
	# 'OperatingCashFlow',
	# 'OperatingExpense',
	# 'ReconciledCostOfRevenue',
	# 'CostOfRevenue',
	# 'GrossProfit',
	'EndCashPosition',
	# 'TotalExpenses',
	# 'TotalRevenue',
	# 'MarketCap',
	# 'EnterpriseValue'
]

df2 = df[['Quarter'] + measures + [x + 'YoY' for x in measures]]
df2.to_csv('{}/{}_test.csv'.format(data_dir, stock), index=False)
